In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

615179.ipynb  data  external-libraries	work


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     |████████████████████████████████| 122kB 411kB/s eta 0:00:01


In [1]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sbn
import time
import math
from sklearn.preprocessing import LabelEncoder


In [ ]:
train=pd.read_csv('./data/data42684/train.csv')
test=pd.read_csv('./data/data42684/test.csv')
songs=pd.read_csv('./data/data42684/songs.csv')
song_extra=pd.read_csv('./data/data42684/song_extra.csv')
members=pd.read_csv('./data/data42684/members.csv')

In [ ]:
train.shape+test.shape+songs.shape+song_extra.shape+members.shape

(7377418, 6, 2556790, 6, 2296320, 7, 2295971, 3, 34403, 7)

In [ ]:
train_v=train[math.ceil(train.shape[0]*0.8):]
train_t=train[0:math.ceil(train.shape[0]*0.8)]
train_v.to_csv('./data/training/validation.csv',index=False)
train_t.to_csv('./data/training/train.csv',index=False)


In [ ]:
train_v.shape+train_t.shape

(1475483, 6, 5901935, 6)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sbn
import time
import math
from sklearn.preprocessing import LabelEncoder

In [ ]:
train=pd.read_csv('./data/training/train.csv')
valid_test=pd.read_csv('./data/training/validation.csv')
test=pd.read_csv('./data/data42684/test.csv')
songs=pd.read_csv('./data/data42684/songs.csv')
song_extra=pd.read_csv('./data/data42684/song_extra.csv')
members=pd.read_csv('./data/data42684/members.csv')

In [ ]:
song_id_set=set(train['song_id'].append(valid_test['song_id']).append(test['song_id']))#如果把测试集一块分析的话要加入test.csv
songs['appeared']=songs['song_id'].apply(lambda x: True if x in song_id_set else False)
songs=songs[songs.appeared]
songs.drop('appeared',axis=1,inplace=True)

song_extra['appeared']=song_extra['song_id'].apply(lambda x: True if x in song_id_set else False)
song_extra=song_extra[song_extra.appeared]
song_extra.drop('appeared',axis=1,inplace=True)

msno_set=set(train['msno'].append(valid_test['msno']).append(test['msno']))#如果把测试集一块分析的话要加入test.csv
members['appeared']=members['msno'].apply(lambda x: True if x in msno_set else False)
members=members[members.appeared]
members.drop('appeared',axis=1,inplace=True)


In [ ]:
songs.shape+song_extra.shape+members.shape

(419781, 7, 419661, 3, 34403, 7)

In [ ]:
members['bd']=members['bd'].apply(lambda x: np.nan if x<=0 or x>=95 else x)
members['bd_missing']=(members['bd'].isnull())*1.0

In [ ]:
songs.isna().sum()

song_id             0
song_length         0
genre_ids        9005
artist_name         0
composer       182563
lyricist       316755
language            1
dtype: int64

In [ ]:
#songs中的缺失值
songs['genre_ids'].fillna('0',inplace=True)
songs['artist_name'].fillna('no_artist_name',inplace=True)
songs['composer'].fillna('no_composer',inplace=True)
songs['lyricist'].fillna('no_lyricist',inplace=True)

#填补language中的缺失值
#songs_temp=songs.loc[songs['language'].isnull()]
# language_value = (songs.loc[songs['artist_name']==(songs_temp['artist_name'].values)[0]]['language'][0:1].values)[0]
# songs['language'].fillna(language_value,inplace=True)
songs['language'].fillna(songs['language'].mode()[0],inplace=True)

#填补members中缺失值
members['bd'].fillna(members['bd'].median(),inplace=True)
members['gender'].fillna('Other',inplace=True)

#填补训练集中缺失值
train['source_system_tab'].fillna(train['source_system_tab'].mode()[0],inplace=True)
train['source_screen_name'].fillna(train['source_screen_name'].mode()[0],inplace=True)
train['source_type'].fillna(train['source_type'].mode()[0],inplace=True)

#填补验证集中缺失值
valid_test['source_system_tab'].fillna(valid_test['source_system_tab'].mode()[0],inplace=True)
valid_test['source_screen_name'].fillna(valid_test['source_screen_name'].mode()[0],inplace=True)
valid_test['source_type'].fillna(valid_test['source_type'].mode()[0],inplace=True)


In [ ]:
#labelencoder编码
# 对所有的msno做labelEncoder编码
msno_encoder=LabelEncoder()
msno_encoder.fit(members['msno'].values)
members['msno']=msno_encoder.transform(members['msno'])
train['msno']=msno_encoder.transform(train['msno'])
valid_test['msno']=msno_encoder.transform(valid_test['msno'])
test['msno']=msno_encoder.transform(test['msno'])

In [ ]:
# 对所有的song_id做labelEncoder编码
song_id_encoder=LabelEncoder()
song_id_encoder.fit(train['song_id'].append(valid_test['song_id']).append(test['song_id']))#是test还是valid_test
songs['song_id']=song_id_encoder.transform(songs['song_id'])
song_extra['song_id']=song_id_encoder.transform(song_extra['song_id'])
train['song_id']=song_id_encoder.transform(train['song_id'])
valid_test['song_id']=song_id_encoder.transform(valid_test['song_id'])#是test还是valid_test
test['song_id']=song_id_encoder.transform(test['song_id'])

In [ ]:
# 对train和test中其他类别型变量做labelEncoder编码
columns = ['source_system_tab', 'source_screen_name', 'source_type']
for column in columns:
    column_encoder=LabelEncoder()
    column_encoder.fit(train[column].append(valid_test[column]))#是test还是valid_test
    train[column]=column_encoder.transform(train[column])
    valid_test[column]=column_encoder.transform(valid_test[column])#是test还是valid_test

In [ ]:
# 对members中的city、gender、registered_via做labelEncoder编码
columns = ['city', 'gender', 'registered_via']
for column in columns:
    column_encoder=LabelEncoder()
    column_encoder.fit(members[column])#是test还是valid_test
    members[column]=column_encoder.transform(members[column])

处理songs中字段

In [ ]:
# 处理genre_ids, 分割成  first_genre_id
#                      second_genre_id
#                      third_genre_id
# 统计一首歌出现的流派数目 genre_id_cnt
genre_id=np.zeros((len(songs),4))
for i in range(len(songs)):
    ids=str(songs['genre_ids'].values[i]).split('|')
    if len(ids)>2:
        genre_id[i,0]=int(ids[0])
        genre_id[i,1]=int(ids[1])
        genre_id[i,2]=int(ids[2])
    elif len(ids)>1:
        genre_id[i,0]=int(ids[0])
        genre_id[i,1]=int(ids[1])
    elif len(ids)==1:
        genre_id[i,0]=int(ids[0])
    genre_id[i,3]=int(len(ids))
songs['first_genre_id']=genre_id[:,0]
songs['second_genre_id']=genre_id[:,1]
songs['third_genre_id']=genre_id[:,2]
songs['genre_id_cnt']=genre_id[:,3]
genre_encoder=LabelEncoder()
genre_encoder.fit(songs.first_genre_id.append(songs.second_genre_id).append(songs.third_genre_id))
songs['first_genre_id']=genre_encoder.transform(songs['first_genre_id'])
songs['second_genre_id']=genre_encoder.transform(songs['second_genre_id'])
songs['third_genre_id']=genre_encoder.transform(songs['third_genre_id'])
songs.drop('genre_ids',axis=1,inplace=True)

In [ ]:
#统计每首歌的歌手数目
def artist_count(x):
    return x.count('and')+x.count(',')+x.count(' feat')+x.count('&')+1
songs['artist_cnt']=songs['artist_name'].apply(artist_count).astype(np.int64)
def get_count(x):
    try:
        return sum(map(x.count,['|','/','\\',';']))+1
    except:
        return 0
songs['lyricist_cnt']=songs['lyricist'].apply(get_count).astype(np.int64)
songs['composer_cnt']=songs['composer'].apply(get_count).astype(np.int64)
songs['is_featured']=songs['artist_name'].apply(lambda x: 1 if ' feat' in str(x) else 0).astype(np.int64)

In [ ]:
# 歌曲的演唱者只保留第一个
def get_first_artist(x):
    if x.count('and')>0:
        x=x.split('and')[0]
    if x.count(',')>0:
        x=x.split(',')[0]
    if x.count(' feat')>0:
        x=x.split(' feat')[0]
    if x.count('&')>0:
        x=x.split('&')[0]
    return x.strip()
songs['artist_name']=songs['artist_name'].apply(get_first_artist)

In [ ]:
# 歌曲的作词人和作曲人只保留第一个
def get_first_term(x):
    if x.count('|')>0:
        x=x.split('|')[0]
    if x.count('/')>0:
        x=x.split('/')[0]
    if x.count('\\')>0:
        x=x.split('\\')[0]
    if x.count(';')>0:
        x=x.split(';')[0]
    return x.strip()
songs['lyricist']=songs['lyricist'].apply(get_first_term)
songs['composer']=songs['composer'].apply(get_first_term)

In [ ]:
# 对处理后的artist_name、composer、lyricist、language做LabelEncoder        
columns = ['artist_name', 'lyricist', 'composer','language']
for column in columns:
    column_encoder=LabelEncoder()
    column_encoder.fit(songs[column])
    songs[column]=column_encoder.transform(songs[column])

In [ ]:
# 按演唱者分类，统计每个演唱者唱过几首歌
artist_song_cnt=songs.groupby(by='artist_name').count()['song_id'].to_dict()
songs['artist_song_cnt']=songs['artist_name'].apply(lambda x:artist_song_cnt[x] if not np.isnan(x) else np.nan)
# 按作曲人分类，统计每个作曲人编曲数目
composer_song_cnt=songs.groupby(by='composer').count()['song_id'].to_dict()
songs['composer_song_cnt']=songs['composer'].apply(lambda x:composer_song_cnt[x] if not np.isnan(x) else np.nan)
# 按作词人分类，统计每个作词人作词数目
lyricist_song_cnt=songs.groupby(by='lyricist').count()['song_id'].to_dict()
songs['lyricist_song_cnt']=songs['lyricist'].apply(lambda x:lyricist_song_cnt[x] if not np.isnan(x) else np.nan)
# 按歌曲风格分类，统计每个流派包含多少首歌
genre_song_cnt=songs.groupby(by='first_genre_id').count()['song_id'].to_dict()
songs['genre_song_cnt']=songs['first_genre_id'].apply(lambda x:genre_song_cnt[x] if not np.isnan(x) else np.nan)

处理song_extra中的字段

In [ ]:
# 歌曲出版的国家码 -- cn
# 歌曲出版者码    -- xxx
# 歌曲录制年份    -- year 
data = train[['msno', 'song_id']].append(valid_test[['msno', 'song_id']]).append(test[['msno', 'song_id']])
songs = songs.merge(song_extra, on='song_id', how='left')

isrc = songs['isrc']
songs['cn'] = isrc.str.slice(0, 2)
songs['xxx'] = isrc.str.slice(2, 5)
songs['year'] = isrc.str.slice(5, 7).astype(float)
# 歌曲录制的年份转换为4位
songs['year'] = songs['year'].apply(lambda x: 2000+x if x < 18 else 1900+x)

# 增加一个新特征，代表是否isrc缺失
songs['isrc_missing'] = (songs['cn'].isnull()) * 1.0
songs['cn'] = LabelEncoder().fit_transform(songs['cn'].fillna('None'))
songs['xxx'] = LabelEncoder().fit_transform(songs['xxx'].fillna('None'))
songs['year'].fillna(songs['year'].median(),inplace=True)

# 按国家码分类，统计每个国家码的歌曲数目
song_cn_cnt = songs.groupby(by='cn').count()['song_id'].to_dict()
songs['cn_song_cnt'] = songs['cn'].apply(lambda x: song_cn_cnt[x] if not np.isnan(x) else None)

# 按出版码分类，统计每个出版者出版的歌曲数目
song_xxx_cnt = songs.groupby(by='xxx').count()['song_id'].to_dict()
songs['xxx_song_cnt'] = songs['xxx'].apply(lambda x: song_xxx_cnt[x] if not np.isnan(x) else None)

# 按歌曲年份分类，统计每个年份录制的歌曲数目
song_year_cnt = songs.groupby(by='year').count()['song_id'].to_dict()
songs['year_song_cnt'] = songs['year'].apply(lambda x: song_year_cnt[x] if not np.isnan(x) else None)

data = data.merge(songs, on='song_id', how='left')
# 按国家码分类，统计每个国家码下有多少用户收听
song_cn_cnt = data.groupby(by='cn').count()['msno'].to_dict()
songs['cn_rec_cnt'] = songs['cn'].apply(lambda x: song_cn_cnt[x] if not np.isnan(x) else None)

# 按出版码分类，统计每个出版码下有多少用户收听
song_xxx_cnt = data.groupby(by='xxx').count()['msno'].to_dict()
songs['xxx_rec_cnt'] = songs['xxx'].apply(lambda x: song_xxx_cnt[x] if not np.isnan(x) else None)

# 按歌曲年份分类，统计每个年份下有多少用户收听
song_year_cnt = data.groupby(by='year').count()['msno'].to_dict()
songs['year_rec_cnt'] = songs['year'].apply(lambda x: song_year_cnt[x] if not np.isnan(x) else None)

features = ['cn_song_cnt', 'xxx_song_cnt', 'year_song_cnt', 'cn_rec_cnt', 'xxx_rec_cnt', 'year_rec_cnt']
for feat in features:
    songs[feat] = np.log1p(songs[feat])

songs.drop(['name', 'isrc'], axis=1, inplace=True)

In [ ]:
# 按歌曲id分类，统计每首歌被播放的用户数目
song_rec_cnt=data.groupby(by='song_id').count()['msno'].to_dict()
songs['song_rec_cnt']=songs['song_id'].apply(lambda x:song_rec_cnt[x] if not np.isnan(x) else np.nan)
# 按演唱者分类，统计每个演唱者被收听过的用户数目
artist_rec_cnt=data.groupby(by='artist_name').count()['msno'].to_dict()
songs['artist_rec_cnt']=songs['artist_name'].apply(lambda x:artist_rec_cnt[x] if not np.isnan(x) else np.nan)
# 按作曲人分类，统计每个作曲人被收听过的用户数目
composer_rec_cnt=data.groupby(by='composer').count()['msno'].to_dict()
songs['composer_rec_cnt']=songs['composer'].apply(lambda x:composer_rec_cnt[x] if not np.isnan(x) else np.nan)
# 按作词人分类，统计每个作词人被收听的用户数目
lyricist_rec_cnt=data.groupby(by='lyricist').count()['msno'].to_dict()
songs['lyricist_rec_cnt']=songs['lyricist'].apply(lambda x:lyricist_rec_cnt[x] if not np.isnan(x) else np.nan)
# 按first_genre_id流派分类，统计每个风格被收听的用户数目
genre_rec_cnt=data.groupby(by='first_genre_id').count()['msno'].to_dict()
songs['genre_rec_cnt']=songs['first_genre_id'].apply(lambda x:genre_rec_cnt[x] if not np.isnan(x) else np.nan)


In [ ]:
features = ['song_length', 'song_rec_cnt', 'artist_song_cnt', 'composer_song_cnt', 
        'lyricist_song_cnt', 'genre_song_cnt', 'artist_rec_cnt', 
        'composer_rec_cnt', 'lyricist_rec_cnt', 'genre_rec_cnt']
for feat in features:
    songs[feat]=np.log1p(songs[feat])


In [ ]:
# count：训练集里每首歌曲被听取的次数，即该歌曲在数据集中出现的总次数
# mean：被重复听取的概率（sum(target=1)/count，即一歌曲被用户第二次播放/该歌曲在数据集
train_merge_songs=train[['msno','song_id','target']].append(valid_test[['msno','song_id','target']]).merge(songs,on='song_id')#valid_test or test
song_mean_count=train_merge_songs[['song_id','target']].groupby(['song_id']).agg(['mean','count'])
song_mean_count.reset_index(inplace=True)
song_mean_count.columns=list(map(''.join,song_mean_count.columns.values))
song_mean_count.columns=['song_id','repeat_play_chance','plays']
songs=songs.merge(song_mean_count,on='song_id',how='right')

处理members中的字段

In [ ]:
data=train[['msno','song_id']].append(valid_test[['msno','song_id']]).append(test[['msno','song_id']])#是valid_test还是test
# 统计一个用户听过多少首歌(训练集+测试集)
mem_rec_cnt=data.groupby(by='msno').count()['song_id'].to_dict()
members['mem_rec_cnt']=members['msno'].apply(lambda x:mem_rec_cnt[x])
members['mem_rec_cnt']=np.log1p(members['mem_rec_cnt'])

In [ ]:
# 到期时间减去注册时间，得到用户的会员时间，作为新特征加入members
members['expiration_date']=members['expiration_date'].astype(np.str)
members['registration_init_time']=members['registration_init_time'].astype(np.str)
members['membership_days']=pd.to_datetime(members['expiration_date']).subtract(pd.to_datetime(members['registration_init_time'])).dt.days.astype(int)
# 将registration_init_time拆分成年、月、日，并作为新特征加入到menmbers中
members['registration_year']=pd.to_datetime(members['registration_init_time']).dt.year
members['registration_month']=pd.to_datetime(members['registration_init_time']).dt.month
members['registration_day']=pd.to_datetime(members['registration_init_time']).dt.day
# 将expiration_date拆分成年、月、日，并作为新特征加入到members中
members['expiration_year']=pd.to_datetime(members['expiration_date']).dt.year
members['expiration_month']=pd.to_datetime(members['expiration_date']).dt.month
members['expiration_day']=pd.to_datetime(members['expiration_date']).dt.day
members['registration_init_time']=members['registration_init_time'].apply(lambda x: time.mktime(time.strptime(str(x),'%Y%m%d')))
members['expiration_date']=members['expiration_date'].apply(lambda x: time.mktime(time.strptime(str(x),'%Y%m%d')))
members=members.drop(['registration_init_time','expiration_date'],axis=1)

In [ ]:
# 对train和test中的相关source的三个字段做独热编码, 并计算相关概率
dummy_feat = ['source_system_tab', 'source_screen_name', 'source_type']
concat_train_test=train.drop('target',axis=1).append(valid_test.drop('target',axis=1))
for feat in dummy_feat:
    dummies=pd.get_dummies(concat_train_test[feat])
    dummies.columns=['msno_%s_'%feat+'%s'%col for col in dummies.columns]
    dummies['msno']=concat_train_test['msno'].values
    dummies=dummies.groupby('msno').mean()
    dummies.reset_index()
    members=members.merge(dummies,on='msno',how='left')

In [ ]:
train.to_csv('./data/training/train_validation_r1.csv',index=False)
valid_test.to_csv('./data/training/test_validation_r1.csv',index=False)
members.to_csv('./data/training/members_validation_r1.csv',index=False)
songs.to_csv('./data/training/songs_validation_r1.csv',index=False)

增加后验概率

In [ ]:
# 把用户的所有特征合并到训练集和验证集中
train_temp=train.merge(members,on='msno',how='left')
valid_test_temp=valid_test.merge(members,on='msno',how='left')
# 查找训练集和验证集中包含‘source_system_tab’字符的特征
train_source_system_tab=train_temp[[col for col in train_temp.columns if 'source_system_tab' in col]]
valid_test_source_system_tab=valid_test_temp[[col for col in valid_test_temp.columns if 'source_system_tab' in col]]
# 查找训练集和验证集中包含‘source_screen_name’字符的特征
train_source_screen_name=train_temp[[col for col in train_temp.columns if 'source_screen_name' in col]]
valid_test_source_screen_name=valid_test_temp[[col for col in valid_test_temp.columns if 'source_screen_name' in col]]
# 查找训练集和验证集中包含‘source_type’字符的特征
train_source_type=train_temp[[col for col in train_temp.columns if 'source_type' in col]]
valid_test_source_type=valid_test_temp[[col for col in valid_test_temp.columns if 'source_type' in col]]

In [ ]:
train['msno_source_system_tab_prob']=train_source_system_tab.apply(lambda x:x['msno_source_system_tab_%d'%x['source_system_tab']],axis=1)
valid_test['msno_source_system_tab_prob']=valid_test_source_system_tab.apply(lambda x:x['msno_source_system_tab_%d'%x['source_system_tab']],axis=1)
train['msno_source_screen_name_prob']=train_source_screen_name.apply(lambda x:x['msno_source_screen_name_%d'%x['source_screen_name']],axis=1)
valid_test['msno_source_screen_name_prob']=valid_test_source_screen_name.apply(lambda x:x['msno_source_screen_name_%d'%x['source_screen_name']],axis=1)
train['msno_source_type_prob']=train_source_type.apply(lambda x:x['msno_source_type_%d'%x['source_type']],axis=1)
valid_test['msno_source_type_prob']=valid_test_source_type.apply(lambda x:x['msno_source_type_%d'%x['source_type']],axis=1)

In [ ]:
train.to_csv('./work/training/train_validation_prob.csv',index=False)
valid_test.to_csv('./work/training/valid_test_validation_prob.csv',index=False)
members.to_csv('./work/training/members_validation_prob.csv',index=False)
songs.to_csv('./work/training/songs_validation_prob.csv',index=False)

用户-歌曲关系矩阵分解、用户-歌手关系矩阵分解

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sbn
import time
import math
from sklearn.preprocessing import LabelEncoder
from scipy import sparse
from scipy.sparse.linalg import svds

In [ ]:
train=pd.read_csv('work/training/train_validation_prob.csv')
valid_test=pd.read_csv('work/training/valid_test_validation_prob.csv')
members=pd.read_csv('work/training/members_validation_prob.csv')
songs=pd.read_csv('work/training/songs_validation_prob.csv')


In [ ]:
concat = train[['msno', 'song_id']].append(valid_test[['msno', 'song_id']])
member_cnt = concat['msno'].max() + 1
song_cnt = concat['song_id'].max() + 1
artist_cnt = int(songs['artist_name'].max() + 1)
print(len(concat))

7377418


In [ ]:
# 设计用户-歌曲关系矩阵，并做SVD分解成三个矩阵
n_component = 48

data = np.ones(len(concat))
msno = concat['msno'].values
song_id = concat['song_id'].values

rating = sparse.coo_matrix((data, (msno, song_id)))
rating = (rating > 0) * 1.0

[u, s, vt] = svds(rating, k=n_component)
print(s[::-1])
s_song = np.diag(s[::-1])

# 保留跟用户相关的48维特征，存入members表中
members_topics = pd.DataFrame(u[:, ::-1])
members_topics.columns = ['member_component_%d'%i for i in range(n_component)]
members_topics['msno'] = range(member_cnt)
members = members.merge(members_topics, on='msno', how='right')

# 保留跟歌曲相关的48维特征，存入songs表中
song_topics = pd.DataFrame(vt.transpose()[:, ::-1])
song_topics.columns = ['song_component_%d'%i for i in range(n_component)]
song_topics['song_id'] = range(song_cnt)
songs = songs.merge(song_topics, on='song_id', how='right')




[929.19644441 348.80981297 309.40275499 252.8020833  235.66103789
 184.89734987 178.8854274  163.36788383 158.45152867 151.20234554
 147.54990949 146.69235095 141.14892669 132.07849752 129.90593115
 125.32091035 122.28441182 119.65531332 117.63870037 115.73739365
 113.58651385 112.08390834 111.2969255  109.59026882 108.62116088
 106.05728495 105.57023407 102.20677814 101.28490626 100.46536012
  97.84094794  96.92911592  96.04246875  95.87283229  93.84138743
  92.73973012  91.85405027  90.3989575   89.84799528  89.29548723
  88.31527645  88.11258867  86.453424    86.01874769  84.99338776
  84.59675956  83.97524489  83.40244589]


In [ ]:
# 设计用户-歌曲关系矩阵，并做SVD分解成三个矩阵 
n_component = 16

concat = concat.merge(songs[['song_id', 'artist_name']], on='song_id', how='left')
concat = concat[concat['artist_name'] >= 0]
msno = concat['msno'].values
artist = concat['artist_name'].values.astype(int)

print(len(concat))
data = np.ones(len(concat))
rating_tmp = sparse.coo_matrix((data, (msno, artist)))

rating = np.log1p(rating_tmp) * 0.3 + (rating_tmp > 0) * 1.0

[u, s, vt] = svds(rating, k=n_component)
print(s[::-1])
s_artist = np.diag(s[::-1])

# 保留跟用户相关的16维特征，存入members表中
members_topics = pd.DataFrame(u[:, ::-1])
members_topics.columns = ['member_artist_component_%d'%i for i in range(n_component)]
members_topics['msno'] = range(member_cnt)
members = members.merge(members_topics, on='msno', how='left')

# 保留跟artist name相关的16维特征，存入songs表中
artist_topics = pd.DataFrame(vt.transpose()[:, ::-1])
artist_topics.columns = ['artist_component_%d'%i for i in range(n_component)]
artist_topics['artist_name'] = range(artist_cnt)
songs = songs.merge(artist_topics, on='artist_name', how='left')


7377304
[1231.90585798  405.63937345  303.7062179   278.05664389  243.94539539
  214.25753678  175.36567621  172.69229575  157.80083028  153.16554079
  149.3448921   146.934055    138.7035043   135.44997276  132.95733285
  126.14460143]


In [ ]:
# 增加新特征
members = members.sort_values(by='msno')
songs = songs.sort_values(by='song_id')

mem_cols = ['member_component_%d'%i for i in range(48)]
song_cols = ['song_component_%d'%i for i in range(48)]
members.columns
member_embeddings = members[mem_cols].values
song_embeddings = songs[song_cols].values

mem_cols = ['member_artist_component_%d'%i for i in range(16)]
song_cols = ['artist_component_%d'%i for i in range(16)]

member_artist_embeddings = members[mem_cols].values
song_artist_embeddings = songs[song_cols].values

train_dot = np.zeros((len(train), 2))
valid_test_dot = np.zeros((len(valid_test), 2))

for i in range(len(train)):
    msno_idx = train['msno'].values[i]
    song_idx = train['song_id'].values[i]
    
    train_dot[i, 0] = np.dot(member_embeddings[msno_idx], np.dot(s_song, song_embeddings[song_idx]))
    train_dot[i, 1] = np.dot(member_artist_embeddings[msno_idx], np.dot(s_artist, song_artist_embeddings[song_idx]))

for i in range(len(valid_test)):
    msno_idx = valid_test['msno'].values[i]
    song_idx = valid_test['song_id'].values[i]
    
    valid_test_dot[i, 0] = np.dot(member_embeddings[msno_idx], np.dot(s_song, song_embeddings[song_idx]))
    valid_test_dot[i, 1] = np.dot(member_artist_embeddings[msno_idx], np.dot(s_artist, song_artist_embeddings[song_idx]))

train['song_embeddings_dot'] = train_dot[:, 0]
train['artist_embeddings_dot'] = train_dot[:, 1]

valid_test['song_embeddings_dot'] = valid_test_dot[:, 0]
valid_test['artist_embeddings_dot'] = valid_test_dot[:, 1]

In [ ]:
# 保存SVD结果的文件
train.to_csv('work/training/train_validation_svd.csv', index=False)
valid_test.to_csv('work/training/valid_test_validation_svd.csv', index=False)
members.to_csv('work/training/members_validation_svd.csv', index=False)
songs.to_csv('work/training/songs_validation_svd.csv', index=False)

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
train=pd.read_csv('work/training/train_validation_svd.csv')
valid_test=pd.read_csv('work/training/valid_test_validation_svd.csv')
members=pd.read_csv('work/training/members_validation_svd.csv')
songs=pd.read_csv('work/training/songs_validation_svd.csv')

In [ ]:
# 保存数据集
train.to_csv('work/training/train_final.csv',index=False,float_format='%.6f')
valid_test.to_csv('work/training/valid_test_final.csv',index=False,float_format='%.6f')
members.to_csv('work/training/members_gbdt.csv',index=False)

In [ ]:
columns = ['composer', 'lyricist', 'language', 'first_genre_id', 'second_genre_id', 'third_genre_id']
for col in columns:
    songs[col].fillna(0,inplace=True)
    songs[col]=songs[col].astype(int)
songs['artist_name'].fillna((songs['artist_name'].max())+1,inplace=True)
songs['artist_name']=songs['artist_name'].astype(int)
songs['isrc_missing'].fillna(0,inplace=True)
songs['isrc_missing']=songs['isrc_missing'].astype(int)
songs.to_csv('work/training/songs_gbdt.csv',index=False)

In [ ]:
songs['song_id_missiing']=np.isnan(songs['song_length'].values)*1
columns = ['song_length', 'genre_id_cnt', 'artist_cnt', 'lyricist_cnt', 'composer_cnt', 'is_featured',
           'artist_song_cnt', 'composer_song_cnt', 'lyricist_song_cnt', 'genre_song_cnt', 'song_rec_cnt', 
           'artist_rec_cnt', 'composer_rec_cnt', 'lyricist_rec_cnt', 'genre_rec_cnt', 'cn', 'xxx', 'year', 
           'cn_song_cnt', 'xxx_song_cnt', 'year_song_cnt', 'cn_rec_cnt', 'xxx_rec_cnt', 'year_rec_cnt', 
           'repeat_play_chance','plays'] + ['artist_component_%d'%i for i in range(16)]
for col in columns:
    songs[col].fillna(np.nanmean(songs[col]),inplace=True)
songs.to_csv('work/training/songs_validation_nn.csv',index=False)

In [2]:
import gc
import math
import time
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import roc_auc_score

In [3]:
train=pd.read_csv('work/training/train_validation_svd.csv')
valid_test=pd.read_csv('work/training/valid_test_validation_svd.csv')
members=pd.read_csv('work/training/members_validation_svd.csv')
songs=pd.read_csv('work/training/songs_validation_svd.csv')

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_rows', None)

In [4]:
      
      train.shape+valid_test.shape+members.shape+songs.shape

(5901935, 11, 1475483, 11, 34403, 118, 419839, 99)

In [5]:
# 训练集和测试集的数目作为索引
concat=train[['msno','song_id']].append(valid_test[['msno','song_id']])
concat['timestamp']=range(len(concat))
## windows_based count
window_sizes = [10, 25, 500, 5000, 10000, 50000]
msno_list=concat['msno'].values
song_list=concat['song_id'].values
def get_window_cnt(values,idx,window_size):
    lower=max(0,idx-window_size)
    upper=min(len(values),idx+window_size)
    return (values[lower:idx]==values[idx]).sum(),(values[idx:upper]==values[idx]).sum()
# 得到不同窗口的前后相同用户或者歌曲个数
for window_size in window_sizes:
    msno_before_cnt=np.zeros(len(concat))
    song_before_cnt = np.zeros(len(concat))
    msno_after_cnt = np.zeros(len(concat))
    song_after_cnt = np.zeros(len(concat))
    for i in range(len(concat)):
        msno_before_cnt[i],msno_after_cnt[i]=get_window_cnt(msno_list,i,window_size)
        song_before_cnt[i],song_after_cnt[i]=get_window_cnt(song_list,i,window_size)
    concat['msno_%d_before_cnt'%window_size]=msno_before_cnt
    concat['song_%d_before_cnt'%window_size] = song_before_cnt
    concat['msno_%d_after_cnt'%window_size] = msno_after_cnt
    concat['song_%d_after_cnt'%window_size] = song_after_cnt
# 统计当前用户和当前歌曲，相同的样本数
msno_dict = defaultdict(lambda: 0)
song_dict = defaultdict(lambda: 0)

msno_till_now_cnt = np.zeros(len(concat))
song_till_now_cnt = np.zeros(len(concat))
for i in range(len(concat)):
    msno_till_now_cnt[i] = msno_dict[msno_list[i]]
    msno_dict[msno_list[i]] += 1
    
    song_till_now_cnt[i] = song_dict[song_list[i]]
    song_dict[song_list[i]] += 1

concat['msno_till_now_cnt'] = msno_till_now_cnt
concat['song_till_now_cnt'] = song_till_now_cnt


In [9]:
features = ['msno_till_now_cnt', 'song_till_now_cnt']
for window_size in window_sizes:
    features += ['msno_%d_before_cnt'%window_size, 'song_%d_before_cnt'%window_size, 
            'msno_%d_after_cnt'%window_size, 'song_%d_after_cnt'%window_size]
for feat in features:
    concat[feat] = np.log1p(concat[feat])

# 得到增加特征后的训练集和测试集
#features = ['timestamp'] + features
data = concat[features].values

for i in range(len(features)):
    train[features[i]] = data[:len(train), i]
    valid_test[features[i]] = data[len(train):, i]

In [11]:
# 保存中间结果
train.to_csv('work/training/train_svd_.csv',index=False)
valid_test.to_csv('work/training/valid_test_svd_.csv',index=False)
members.to_csv('work/training/members_svd_.csv',index=False)
songs.to_csv('work/training/songs_svd_.csv',index=False)

In [6]:
time.asctime(time.localtime(1484236800.0))

'Fri Jan 13 00:00:00 2017'

In [7]:
time.asctime(time.localtime(1471190400.0))

'Mon Aug 15 00:00:00 2016'

In [8]:
# 计算当前记录所处的相对时间点
def timestamp_map(x):
    x = (x - 0.0) / (7377417.0 - 0.0) * (1484236800.0 - 1471190400.0) + 1471190400.0
    return x
    
concat['timestamp'] = concat['timestamp'].apply(timestamp_map)

# 增加新特征，每个用户收听歌曲的时间点的平均值
msno_mean = concat.groupby(by='msno').mean()['timestamp'].to_dict()
members['msno_timestamp_mean'] = members['msno'].apply(lambda x: msno_mean[x] if x in msno_mean else concat['timestamp'].mode()[0])

# 增加新特征，每个用户收听歌曲的时间点的标准差。值越大说明用户收听的跨度越大
msno_std = concat.groupby(by='msno').std()['timestamp'].to_dict()
members['msno_timestamp_std'] = members['msno'].apply(lambda x: msno_std[x])

# 增加新特征，每首歌被收听的时间点的平均值
song_mean = concat.groupby(by='song_id').mean()['timestamp'].to_dict()
songs['song_timestamp_mean'] = songs['song_id'].apply(lambda x: song_mean[x])

# 增加新特征，每首歌被收听时间点的标准差。值越大，说明歌曲被收听的时间跨度大
song_std = concat.groupby(by='song_id').std()['timestamp'].to_dict()
songs['song_timestamp_std'] = songs['song_id'].apply(lambda x: song_std[x])

print('Timestamp done.')

features = ['msno_till_now_cnt', 'song_till_now_cnt']
for window_size in window_sizes:
    features += ['msno_%d_before_cnt'%window_size, 'song_%d_before_cnt'%window_size, 
            'msno_%d_after_cnt'%window_size, 'song_%d_after_cnt'%window_size]
for feat in features:
    concat[feat] = np.log1p(concat[feat])

# 得到增加特征后的训练集和测试集
features = ['timestamp'] + features
data = concat[features].values

for i in range(len(features)):
    train[features[i]] = data[:len(train), i]
    test[features[i]] = data[len(train):, i]

KeyError: 2

In [12]:
import gc
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_auc_score

In [13]:
train=pd.read_csv('work/training/train_svd_.csv')
valid_test=pd.read_csv('work/training/valid_test_svd_.csv')
members=pd.read_csv('work/training/members_svd_.csv')
songs=pd.read_csv('work/training/songs_svd_.csv')

In [14]:
columns = ['composer', 'lyricist', 'language', 'first_genre_id', 'second_genre_id', 'third_genre_id','isrc_missing']
for col in columns:
    songs[col].fillna(0,inplace=True)
    songs[col]=songs[col].astype(int)
songs['artist_name'].fillna(np.max(songs['artist_name'])+1,inplace=True)
songs['artist_name']=songs['artist_name'].astype(int)
songs['song_id_missing']=np.isnan(songs['song_length'].values)*1
columns = ['song_length', 'genre_id_cnt', 'artist_song_cnt', 'composer_song_cnt', 
       'lyricist_song_cnt', 'genre_song_cnt', 'song_rec_cnt', 
       'artist_rec_cnt', 'composer_rec_cnt', 'lyricist_rec_cnt', 
       'genre_rec_cnt','cn','xxx','year', 'cn_song_cnt', 
       'xxx_song_cnt', 'year_song_cnt', 'cn_rec_cnt', 'xxx_rec_cnt', 
       'year_rec_cnt',  'artist_cnt', 'lyricist_cnt', 
       'composer_cnt', 'is_featured'] + ['artist_component_%d'%i for i in range(16)]
for col in columns:
    songs[col].fillna(np.nanmean(songs[col]),inplace=True)

In [15]:
songs.to_csv('work/music/songs_val_nn.csv',index=False)

In [17]:
#members['msno_timestamp_std'].fillna(np.nanmin(members['msno_timestamp_std']), inplace=True)

concat = train[['msno', 'song_id', 'source_system_tab', 'source_screen_name', 
        'source_type']].append(valid_test[['msno', 'song_id', 'source_system_tab', 
        'source_screen_name', 'source_type']])
concat = concat.merge(songs[['song_id', 'song_length', 'artist_name', 'first_genre_id', 
        'artist_rec_cnt', 'song_rec_cnt', 'artist_song_cnt', 'xxx', 'year', 
        'language']], on='song_id', how='left')
concat['source'] = concat['source_system_tab'] * 10000 + concat['source_screen_name'] * 100 + concat['source_type']
from sklearn.preprocessing import LabelEncoder
concat['source'] = LabelEncoder().fit_transform(concat['source'].values)

mem_add = pd.DataFrame({'msno': range(concat['msno'].max()+1)})
data_avg = concat[['msno', 'song_length', 'artist_song_cnt', 
        'artist_rec_cnt', 'song_rec_cnt', 'year']].groupby('msno').mean()
data_avg.columns = ['msno_'+i+'_mean' for i in data_avg.columns]
# data_avg['msno'] = data_avg.index.values
data_avg.reset_index()
members = members.merge(data_avg, on='msno', how='left')
data_std=concat[['msno', 'song_length', 'artist_song_cnt', 
        'artist_rec_cnt', 'song_rec_cnt', 'year']].groupby('msno').std()
data_std.columns=['msno_'+i+'_std' for i in data_std.columns]
data_std.reset_index()
members=members.merge(data_std,on='msno',how='left')
#计算msno所属的不同artist_name的个数
#按msno分类，统计每个用户听过的歌手个数
artist_msno=concat[['msno', 'artist_name']].groupby('msno').apply(lambda x:len(set(x['artist_name'].values)))
mem_add['artist_msno_cnt']=artist_msno
mem_add['artist_msno_cnt']=np.log1p(mem_add['artist_msno_cnt'])
#计算用户所属的language中不同取值的概率
language_dummy=pd.get_dummies(concat['language'])
language_dummy['msno']=concat['msno'].values
language_prob=language_dummy.groupby('msno').mean()
language_prob.columns=['msno_language_%d'%i for i in language_prob.columns]
language_prob.reset_index()
members=members.merge(language_prob,on='msno',how='left')

In [18]:
# 生成最终用户信息表
members.to_csv('work/music/members_val_nn.csv', index=False)

In [19]:
# 一些歌曲属性
col = ['artist_name', 'first_genre_id', 'xxx', 'language', 'year', 'source']

# 统计msno-artist_name， msno-first_genre_id, msno-xxx,msno-language, msno-year, msno-source数据对在数据集中的出现次数
for feat in col:
    concat['id'] = concat['msno'] * 100000 + concat[feat]
    id_cnt = concat[['msno', 'id']].groupby('id').count().to_dict()['msno']
    concat['msno_'+feat+'_cnt'] = concat['id'].apply(lambda x: id_cnt[x])
# 统计这些数据对在每个用户收听过的总歌曲里的概率，即出现次数/每个用户收听过的歌曲总数
msno_cnt=concat[['msno','song_id']].groupby('msno').count().to_dict()['song_id']
concat['msno_cnt']=concat['msno'].apply(lambda x:msno_cnt[x])
for feat in col:
    concat['msno_'+feat+'_prob']=concat['msno_'+feat+'_cnt']/concat['msno_cnt']
# 统计song_id-source_system_tab，song_id-source_screen_name, song_id-source_type数据对在数据集中的出现次数
cols = ['source_system_tab', 'source_screen_name', 'source_type']
for col in cols:
    concat['id']=concat['song_id']*10000+concat[col]
    id_cnt=concat[['msno','id']].groupby('id').count().to_dict()['msno']
    concat['song_'+col+'_cnt']=concat['id'].apply(lambda x:id_cnt[x])
# 统计每首歌曲被收听的用户总数
song_cnt=concat[['msno','song_id']].groupby('song_id').count().to_dict()['msno']
concat['song_cnt']=concat['song_id'].apply(lambda x:song_cnt[x])
# 统计数据对在每首歌曲被收听的用户总数的概率，即出现次数/每首歌曲被收听的用户总数
for col in cols:
    concat['song_'+col+'_prob']=concat['song_'+col+'_cnt']/concat['song_cnt']
result = concat[['msno_artist_name_prob', 'msno_first_genre_id_prob', 'msno_xxx_prob', 
        'msno_language_prob', 'msno_year_prob', 'song_source_system_tab_prob', 
        'song_source_screen_name_prob', 'song_source_type_prob', 'source', 'msno_source_prob']]
result['source']=result['source'].astype('category')
for col in result.columns:
    train[col]=result[col].values[:len(train)]
    valid_test[col]=result[col].values[len(train):]
train['artist_embeddings_dot'].fillna(train['artist_embeddings_dot'].mean(), inplace=True)
valid_test['artist_embeddings_dot'].fillna(valid_test['artist_embeddings_dot'].mean(), inplace=True)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# 生成最后训练集和验证集/测试集
train.to_csv('work/music/train_val_nn.csv', index=False)
valid_test.to_csv('work/music/valid_test_val_nn.csv', index=False)   

In [3]:
import sys
sys.path.append('/home/aistudio/package')

import gc
import math
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

ModuleNotFoundError: No module named 'lightgbm'

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 